# Perceptron Learning Phishing Email Classification

In [1]:
%run Perceptron.py

In [2]:
from urllib.request import urlopen
import numpy as np
import pandas as pd

## Reading the Data

In [3]:
def parse_row(line, len_row):
    row = np.zeros(len_row)
    for i, value in enumerate(line.split(",")):
        row[i] = int(value)
    return row

num_cols = 0
columns = []
rows = []
len_attr = len('@attribute')


with urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00327/Training%20Dataset.arff") as f:
    for line in f:
        line = line.decode('utf-8')
        if line.startswith(("-1","0","1")):
            rows.append(parse_row(line, num_cols))
        elif line.startswith("@attribute "):
            col_name = line[len_attr:line.find("{}")].strip().split()[0]
            columns.append(col_name)
            num_cols += 1
        
df = pd.DataFrame(data=rows, columns=columns)

df.head()

having_IP_Address  URL_Length  Shortining_Service  having_At_Symbol  \
0               -1.0         1.0                 1.0               1.0   
1                1.0         1.0                 1.0               1.0   
2                1.0         0.0                 1.0               1.0   
3                1.0         0.0                 1.0               1.0   
4                1.0         0.0                -1.0               1.0   

   double_slash_redirecting  Prefix_Suffix  having_Sub_Domain  SSLfinal_State  \
0                      -1.0           -1.0               -1.0            -1.0   
1                       1.0           -1.0                0.0             1.0   
2                       1.0           -1.0               -1.0            -1.0   
3                       1.0           -1.0               -1.0            -1.0   
4                       1.0           -1.0                1.0             1.0   

   Domain_registeration_length  Favicon  ...  popUpWidnow  Iframe  \
0                         -1.0      1.0  ...          1.0     1.0   
1                         -1.0      1.0  ...          1.0     1.0   
2                         -1.0      1.0  ...          1.0     1.0   
3                          1.0      1.0  ...          1.0     1.0   
4                         -1.0      1.0  ...         -1.0     1.0   

   age_of_domain  DNSRecord  web_traffic  Page_Rank  Google_Index  \
0           -1.0       -1.0         -1.0       -1.0           1.0   
1           -1.0       -1.0          0.0       -1.0           1.0   
2            1.0       -1.0          1.0       -1.0           1.0   
3           -1.0       -1.0          1.0       -1.0           1.0   
4           -1.0       -1.0          0.0       -1.0           1.0   

   Links_pointing_to_page  Statistical_report  Result  
0                     1.0                -1.0    -1.0  
1                     1.0                 1.0    -1.0  
2                     0.0                -1.0    -1.0  
3                    -1.0                 1.0    -1.0  
4                     1.0                 1.0     1.0  

[5 rows x 31 columns]

## Initial Summary Statistics

In [13]:
df.shape

(11055, 31)

In [12]:
df.describe().loc[["mean", "std", "50%"]].T.rename(columns={"50%" : "median"})

mean       std  median
having_IP_Address            0.313795  0.949534     1.0
URL_Length                  -0.633198  0.766095    -1.0
Shortining_Service           0.738761  0.673998     1.0
having_At_Symbol             0.700588  0.713598     1.0
double_slash_redirecting     0.741474  0.671011     1.0
Prefix_Suffix               -0.734962  0.678139    -1.0
having_Sub_Domain            0.063953  0.817518     0.0
SSLfinal_State               0.250927  0.911892     1.0
Domain_registeration_length -0.336771  0.941629    -1.0
Favicon                      0.628584  0.777777     1.0
port                         0.728268  0.685324     1.0
HTTPS_token                  0.675079  0.737779     1.0
Request_URL                  0.186793  0.982444     1.0
URL_of_Anchor               -0.076526  0.715138     0.0
Links_in_tags               -0.118137  0.763973     0.0
SFH                         -0.595749  0.759143    -1.0
Submitting_to_email          0.635640  0.772021     1.0
Abnormal_URL                 0.705292  0.708949     1.0
Redirect                     0.115694  0.319872     0.0
on_mouseover                 0.762099  0.647490     1.0
RightClick                   0.913885  0.405991     1.0
popUpWidnow                  0.613388  0.789818     1.0
Iframe                       0.816915  0.576784     1.0
age_of_domain                0.061239  0.998168     1.0
DNSRecord                    0.377114  0.926209     1.0
web_traffic                  0.287291  0.827733     1.0
Page_Rank                   -0.483673  0.875289    -1.0
Google_Index                 0.721574  0.692369     1.0
Links_pointing_to_page       0.344007  0.569944     0.0
Statistical_report           0.719584  0.694437     1.0
Result                       0.113885  0.993539     1.0

In [20]:
df.corr()

having_IP_Address  URL_Length  \
having_IP_Address                     1.000000   -0.052411   
URL_Length                           -0.052411    1.000000   
Shortining_Service                    0.403461   -0.097881   
having_At_Symbol                      0.158699   -0.075108   
double_slash_redirecting              0.397389   -0.081247   
Prefix_Suffix                        -0.005257    0.055247   
having_Sub_Domain                    -0.080745    0.003997   
SSLfinal_State                        0.071414    0.048754   
Domain_registeration_length          -0.022739   -0.221892   
Favicon                               0.087025   -0.042497   
port                                  0.060979    0.000323   
HTTPS_token                           0.363534   -0.089383   
Request_URL                           0.029773    0.246348   
URL_of_Anchor                         0.099847   -0.023396   
Links_in_tags                         0.006212    0.052869   
SFH                                  -0.010962    0.414196   
Submitting_to_email                   0.077989   -0.014457   
Abnormal_URL                          0.336549   -0.106761   
Redirect                             -0.321181    0.046832   
on_mouseover                          0.084059   -0.045103   
RightClick                            0.042881   -0.013613   
popUpWidnow                           0.096882   -0.049381   
Iframe                                0.054694   -0.013838   
age_of_domain                        -0.010446    0.179426   
DNSRecord                            -0.050733   -0.040823   
web_traffic                           0.002922    0.008993   
Page_Rank                            -0.091774    0.183518   
Google_Index                          0.029153    0.002902   
Links_pointing_to_page               -0.339065   -0.022987   
Statistical_report                   -0.019103   -0.067153   
Result                                0.094160    0.057430   

                             Shortining_Service  having_At_Symbol  \
having_IP_Address                      0.403461          0.158699   
URL_Length                            -0.097881         -0.075108   
Shortining_Service                     1.000000          0.104447   
having_At_Symbol                       0.104447          1.000000   
double_slash_redirecting               0.842796          0.086960   
Prefix_Suffix                         -0.080471         -0.011726   
having_Sub_Domain                     -0.041916         -0.058976   
SSLfinal_State                        -0.061426          0.031220   
Domain_registeration_length            0.060923          0.015522   
Favicon                                0.006101          0.304899   
port                                   0.002201          0.364891   
HTTPS_token                            0.757838          0.104561   
Request_URL                           -0.037235          0.027909   
URL_of_Anchor                          0.000561          0.057914   
Links_in_tags                         -0.133379         -0.070861   
SFH                                   -0.022723         -0.008672   
Submitting_to_email                    0.049328          0.370123   
Abnormal_URL                           0.739290          0.203945   
Redirect                              -0.534530         -0.028160   
on_mouseover                           0.062383          0.279697   
RightClick                             0.038118          0.219503   
popUpWidnow                            0.036616          0.290893   
Iframe                                 0.016581          0.284410   
age_of_domain                         -0.052596         -0.005499   
DNSRecord                              0.436064         -0.047872   
web_traffic                           -0.047074          0.032918   
Page_Rank                              0.014591         -0.064735   
Google_Index                           0.155844          0.037061   
Links_pointing_to_page                -0.19

For specific information on how these categorical variables were determined, refer to the "Phishing Website Features" file.

In [ ]:
num_examples = data.shape[0]
num_features = len(meta.names()) - 1
data = np.array(data.tolist(), dtype=np.int8)
np.random.shuffle(data)
train_examples = round(num_examples*.7)
test_examples = ((num_examples - train_examples) // 2)
train_labels = data[:train_examples,30]
dev_labels = data[train_examples:train_examples+test_examples,30]
test_labels = data[train_examples+test_examples:,30]
train_data = data[:train_examples,:30]
dev_data = data[train_examples:train_examples+test_examples,:30]
test_data = data[train_examples+test_examples:,:30]

In [ ]:
PhishingSimplePerceptron = SimplePerceptron(num_features, epochs=100)
SimpleWeights = PhishingSimplePerceptron.train(train_data, train_labels, dev_data, dev_labels)[0]

In [ ]:
PhishingDynamicPerceptron = DynamicPerceptron(num_features, epochs=100)
DynamicWeights = PhishingDynamicPerceptron.train(train_data, train_labels)[0]

In [ ]:
PhishingMarginPerceptron = MarginPerceptron(num_features, epochs=100)
MarginWeights = PhishingMarginPerceptron.train(train_data, train_labels)[0]

In [ ]:
PhishingAveragedPerceptron = AveragedPerceptron(num_features, epochs=100)
AveragedWeights = PhishingAveragedPerceptron.train(train_data, train_labels)[0]

In [ ]:
PhishingAggressiveMarginPerceptron = AggressiveMarginPerceptron(num_features, epochs=100)
AggressiveMarginWeights = PhishingAggressiveMarginPerceptron.train(train_data, train_labels)[0]

In [ ]:
PhishingSimplePerceptron.test(test_data, test_labels)
PhishingDynamicPerceptron.test(test_data, test_labels)
PhishingMarginPerceptron.test(test_data, test_labels)
PhishingAveragedPerceptron.test(test_data, test_labels)
PhishingAggressiveMarginPerceptron.test(test_data, test_labels)